In [ ]:
-- 1. All instructors teaching in multiple departments
SELECT
    i.FirstName,
    i.LastName,
    COUNT(DISTINCT d.DepartmentID) AS DepartmentCount
FROM ClassSchedule.Instructor i
JOIN ClassSchedule.InstructorDepartment id
    ON i.InstructorID = id.InstructorID
JOIN ClassSchedule.Department d
    ON id.DepartmentID = d.DepartmentID
GROUP BY i.FirstName, i.LastName
HAVING COUNT(DISTINCT d.DepartmentID) > 1;

In [ ]:
-- 2. How many instructors are in each department
SELECT
    d.DepartmentCode,
    COUNT(DISTINCT id.InstructorID) AS InstructorCount
FROM ClassSchedule.Department d
LEFT JOIN ClassSchedule.InstructorDepartment id
    ON d.DepartmentID = id.DepartmentID
GROUP BY d.DepartmentCode;

In [ ]:
-- 3. Classes by course with enrollment for the semester
SELECT
    crs.CourseCode,
    crs.CourseName,
    c.Semester,
    COUNT(c.ClassID) AS ClassCount,
    SUM(c.Enrolled) AS TotalEnrolled,
    SUM(c.EnrollmentLimit) AS TotalCapacity,
    CAST(
        (SUM(c.Enrolled) * 100.0) / NULLIF(SUM(c.EnrollmentLimit), 0)
        AS DECIMAL(5,2)
    ) AS EnrollmentPercentage
FROM ClassSchedule.Class c
JOIN ClassSchedule.Course crs
    ON c.CourseID = crs.CourseID
GROUP BY
    crs.CourseCode,
    crs.CourseName,
    c.Semester
ORDER BY crs.CourseCode;

In [ ]:
-- 4. Each class belongs to exactly one course
SELECT
    c.ClassID,
    crs.CourseCode,
    crs.CourseName
FROM ClassSchedule.Class c
JOIN ClassSchedule.Course crs
    ON c.CourseID = crs.CourseID;

In [ ]:
-- 5. Each class is taught by exactly one instructor
SELECT
    c.ClassID,
    i.FirstName,
    i.LastName
FROM ClassSchedule.Class c
JOIN ClassSchedule.Instructor i
    ON c.InstructorID = i.InstructorID;

In [ ]:
-- 6. Departments and their multiple courses
SELECT
    d.DepartmentCode,
    COUNT(crs.CourseID) AS CourseCount
FROM ClassSchedule.Department d
JOIN ClassSchedule.Course crs
    ON d.DepartmentID = crs.DepartmentID
GROUP BY d.DepartmentCode;

In [ ]:
-- 7. Classes with physical rooms
SELECT
    c.ClassID,
    c.Semester,
    c.Section,
    r.RoomNumber
FROM ClassSchedule.Class c
LEFT JOIN ClassSchedule.Room r
    ON c.RoomID = r.RoomID;

In [ ]:
-- 8. Courses with multiple sections in the same semester
SELECT
    crs.CourseCode,
    c.Semester,
    COUNT(c.ClassID) AS SectionCount
FROM ClassSchedule.Class c
JOIN ClassSchedule.Course crs
    ON c.CourseID = crs.CourseID
GROUP BY crs.CourseCode, c.Semester;

In [ ]:
-- 9. Rooms with multiple classes in the semester
SELECT
    bl.BuildingCode,
    r.RoomNumber,
    c.Semester,
    COUNT(c.ClassID) AS ClassCount
FROM ClassSchedule.Class c
JOIN ClassSchedule.Room r
    ON c.RoomID = r.RoomID
JOIN ClassSchedule.BuildingLocation bl
    ON r.BuildingLocationID = bl.BuildingLocationID
GROUP BY
    bl.BuildingCode,
    r.RoomNumber,
    c.Semester
HAVING COUNT(c.ClassID) > 1;

In [ ]:
-- 10. Instructors can teach more than one class during a semester.
SELECT
    i.FirstName,
    i.LastName,
    c.Semester,
    COUNT(c.ClassID) AS ClassCount
FROM ClassSchedule.Class c
JOIN ClassSchedule.Instructor i
    ON c.InstructorID = i.InstructorID
GROUP BY
    i.FirstName,
    i.LastName,
    c.Semester
HAVING COUNT(c.ClassID) > 1;

In [ ]:
-- 11. Classes that are online do not require a physical room
SELECT
    m.ModeName,
    COUNT(c.ClassID) AS ClassCount
FROM ClassSchedule.Class c
JOIN ClassSchedule.ModeOfInstruction m
    ON c.ModeOfInstructionID = m.ModeOfInstructionID
WHERE c.RoomID IS NULL
GROUP BY m.ModeName;

In [ ]:
-- 12. The number of classes taught by an instructor can be measured for each semester.
SELECT
    i.FirstName,
    i.LastName,
    c.Semester,
    COUNT(c.ClassID) AS ClassesTaught
FROM ClassSchedule.Class c
JOIN ClassSchedule.Instructor i
    ON c.InstructorID = i.InstructorID
GROUP BY
    i.FirstName,
    i.LastName,
    c.Semester
ORDER BY c.Semester, ClassesTaught DESC;

In [ ]:
-- 13. Some campus buildings are used more heavily for classes than others
SELECT
    bl.BuildingCode,
    COUNT(c.ClassID) AS ClassCount
FROM ClassSchedule.Class c
JOIN ClassSchedule.Room r
    ON c.RoomID = r.RoomID
JOIN ClassSchedule.BuildingLocation bl
    ON r.BuildingLocationID = bl.BuildingLocationID
GROUP BY bl.BuildingCode
ORDER BY ClassCount DESC;